In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_data_path = '/content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/labelled_ble_data.csv'
test_data_path = '/content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/BLE_Test_predict.csv'

In [3]:
# Setup and Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import os
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Input, Multiply, Permute, Reshape, Lambda
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Bidirectional, GRU, Layer
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

✓ All imports successful
Num GPUs Available:  1


In [4]:
class AttentionLayer(Layer):
    """
    Custom Attention Layer for Sequence Models

    This layer learns which timesteps in the sequence are most important
    for the classification task. It computes attention weights and returns
    a weighted sum of the input sequence.
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # input_shape: (batch_size, timesteps, features)
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(1,),
            initializer='zeros',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        # inputs shape: (batch_size, timesteps, features)

        # Compute attention scores: (batch_size, timesteps, 1)
        attention_scores = K.tanh(K.dot(inputs, self.W) + self.b)

        # Apply mask if provided (for padded sequences)
        if mask is not None:
            # Expand mask to match attention_scores shape
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, axis=-1)
            # Set masked positions to very negative value
            attention_scores = attention_scores * mask + (1 - mask) * (-1e10)

        # Compute attention weights: (batch_size, timesteps, 1)
        attention_weights = K.softmax(attention_scores, axis=1)

        # Compute weighted sum: (batch_size, features)
        context_vector = K.sum(inputs * attention_weights, axis=1)

        return context_vector

    def compute_output_shape(self, input_shape):
        # Output shape: (batch_size, features)
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        return super(AttentionLayer, self).get_config()

print("✅ Custom Attention Layer defined")

✅ Custom Attention Layer defined


In [5]:
def set_seeds(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

def create_room_groups(df):
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['room_group'] = (df['room'] != df['room'].shift()).cumsum()
    return df

def create_beacon_count_vectors(df):
    """Aggregates readings into 1s vectors. Handles data with or without 'room_group'."""
    vectors = []
    has_groups = 'room_group' in df.columns  # Check if we are in 'training' mode

    for _, group in df.groupby('timestamp'):
        beacon_counts = group['mac address'].value_counts()
        total_readings = len(group)

        vector = [0.0] * 23
        for beacon_id, count in beacon_counts.items():
            if 1 <= beacon_id <= 23:
                vector[int(beacon_id) - 1] = count / total_readings

        entry = {
            'timestamp': group['timestamp'].iloc[0],
            'room': group['room'].iloc[0],
            'beacon_vector': vector
        }

        if has_groups:
            entry['room_group'] = group['room_group'].iloc[0]

        vectors.append(entry)

    return pd.DataFrame(vectors)

def create_sequences_from_groups(vector_df, min_length=3, max_length=50):
    """Used for Training: Creates clean sequences where the room is constant."""
    sequences = []
    labels = []

    for (room, room_group), group in vector_df.groupby(['room', 'room_group']):
        group = group.sort_values('timestamp').reset_index(drop=False)
        seq_length = len(group)

        if seq_length < min_length:
            continue

        if seq_length > max_length:
            group = group.tail(max_length)

        sequence = [row['beacon_vector'] for _, row in group.iterrows()]
        sequences.append(sequence)
        labels.append(room)

    return sequences, labels

print("✅ Basic functions defined")

✅ Basic functions defined


In [6]:
def build_bidirectional_gru_model_with_deep_attention(input_shape, num_classes):
    """
    Deep Bidirectional GRU Architecture with Attention Mechanism

    Architecture:
    1. Masking layer (handle variable-length sequences)
    2. First Bi-GRU (128 units) with return_sequences=True
    3. Dropout (0.3)
    4. Second Bi-GRU (64 units) with return_sequences=True
    5. Dropout (0.2)
    6. ATTENTION LAYER - learns which timesteps matter most
    7. Dense layer (64 units)
    8. Dropout (0.3)
    9. Dense layer (32 units)
    10. Dropout (0.2)
    11. Output layer (softmax)
    """
    inputs = Input(shape=input_shape, name='input_layer')

    # Masking for padded sequences
    masked = Masking(mask_value=0.0, name='masking')(inputs)

    # First Bi-GRU layer - extracts sequential features
    gru1 = Bidirectional(
        GRU(128, return_sequences=True, name='gru_layer_1'),
        name='bidirectional_gru_1'
    )(masked)
    gru1 = Dropout(0.3, name='dropout_1')(gru1)

    # Second Bi-GRU layer - deeper feature extraction
    gru2 = Bidirectional(
        GRU(64, return_sequences=True, name='gru_layer_2'),
        name='bidirectional_gru_2'
    )(gru1)
    gru2 = Dropout(0.2, name='dropout_2')(gru2)

    # ATTENTION MECHANISM
    # Learns which timesteps are most important after deep feature extraction
    attention_output = AttentionLayer(name='attention_layer')(gru2)

    # Dense layers for classification
    dense1 = Dense(64, activation='relu', name='dense_1')(attention_output)
    dense1 = Dropout(0.3, name='dropout_3')(dense1)

    dense2 = Dense(32, activation='relu', name='dense_2')(dense1)
    dense1 = Dropout(0.2, name='dropout_4')(dense2)

    # Output layer
    outputs = Dense(num_classes, activation='softmax', name='output_layer')(dense1)

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name='Deep_BiGRU_with_Attention')

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

print("✅ Deep Attention-based Bi-GRU model architecture defined")

✅ Deep Attention-based Bi-GRU model architecture defined


In [7]:
def create_extended_multidirectional_windows(vector_df):
    """
    Create 7 types of sliding windows for extended multi-directional prediction

    Directions:
    1. backward_10:  [i-9 to i]     - 10s history, predict at i
    2. centered_10:  [i-4 to i+5]   - 10s centered, predict at i
    3. forward_10:   [i to i+9]     - 10s future, predict at i
    4. backward_15:  [i-14 to i]    - 15s history (more context)
    5. forward_15:   [i to i+14]    - 15s future (earlier transition detection)
    6. asymm_past:   [i-11 to i+3]  - 12s past + 4s future (transition from old room)
    7. asymm_future: [i-3 to i+11]  - 4s past + 12s future (entering new room)

    Returns:
        Dictionary with direction names as keys
        Each contains: (sequences, labels, valid_indices)
    """
    # Ensure chronological order and group by day
    vector_df['dt'] = pd.to_datetime(vector_df['timestamp'])
    vector_df['date'] = vector_df['dt'].dt.date

    results = {
        'backward_10': {'sequences': [], 'labels': [], 'indices': []},
        'centered_10': {'sequences': [], 'labels': [], 'indices': []},
        'forward_10': {'sequences': [], 'labels': [], 'indices': []},
        'backward_15': {'sequences': [], 'labels': [], 'indices': []},
        'forward_15': {'sequences': [], 'labels': [], 'indices': []},
        'asymm_past': {'sequences': [], 'labels': [], 'indices': []},
        'asymm_future': {'sequences': [], 'labels': [], 'indices': []},
    }

    for _, day_group in vector_df.groupby('date'):
        day_group = day_group.sort_values('timestamp').reset_index(drop=True)
        vectors = list(day_group['beacon_vector'])
        rooms = list(day_group['room'])
        n = len(vectors)

        for i in range(n):
            # 1. BACKWARD_10: [i-9, ..., i] predict at i
            if i >= 9:
                window = vectors[i - 9 : i + 1]
                results['backward_10']['sequences'].append(window)
                results['backward_10']['labels'].append(rooms[i])
                results['backward_10']['indices'].append((day_group['date'].iloc[0], i))

            # 2. CENTERED_10: [i-4, ..., i, ..., i+5] predict at i
            if i >= 4 and i + 5 < n:
                window = vectors[i - 4 : i + 6]
                results['centered_10']['sequences'].append(window)
                results['centered_10']['labels'].append(rooms[i])
                results['centered_10']['indices'].append((day_group['date'].iloc[0], i))

            # 3. FORWARD_10: [i, ..., i+9] predict at i
            if i + 9 < n:
                window = vectors[i : i + 10]
                results['forward_10']['sequences'].append(window)
                results['forward_10']['labels'].append(rooms[i])
                results['forward_10']['indices'].append((day_group['date'].iloc[0], i))

            # 4. BACKWARD_15: [i-14, ..., i] predict at i (MORE HISTORY)
            if i >= 14:
                window = vectors[i - 14 : i + 1]
                results['backward_15']['sequences'].append(window)
                results['backward_15']['labels'].append(rooms[i])
                results['backward_15']['indices'].append((day_group['date'].iloc[0], i))

            # 5. FORWARD_15: [i, ..., i+14] predict at i (EARLIER TRANSITION DETECTION)
            if i + 14 < n:
                window = vectors[i : i + 15]
                results['forward_15']['sequences'].append(window)
                results['forward_15']['labels'].append(rooms[i])
                results['forward_15']['indices'].append((day_group['date'].iloc[0], i))

            # 6. ASYMM_PAST: [i-11, ..., i, ..., i+3] predict at i (HEAVY PAST BIAS)
            # Good for detecting we're leaving a room
            if i >= 11 and i + 3 < n:
                window = vectors[i - 11 : i + 4]
                results['asymm_past']['sequences'].append(window)
                results['asymm_past']['labels'].append(rooms[i])
                results['asymm_past']['indices'].append((day_group['date'].iloc[0], i))

            # 7. ASYMM_FUTURE: [i-3, ..., i, ..., i+11] predict at i (HEAVY FUTURE BIAS)
            # Good for detecting we're entering a room
            if i >= 3 and i + 11 < n:
                window = vectors[i - 3 : i + 12]
                results['asymm_future']['sequences'].append(window)
                results['asymm_future']['labels'].append(rooms[i])
                results['asymm_future']['indices'].append((day_group['date'].iloc[0], i))

    return results

print("✅ Extended multi-directional window function defined (7 directions)")

✅ Extended multi-directional window function defined (7 directions)


In [8]:
def train_ensemble_models(train_df, n_models=5, base_seed=42, verbose=True):
    """
    Train multiple models with ATTENTION mechanism for deployment

    Returns:
        models: List of trained Keras models (with attention)
        label_encoder: Fitted label encoder
    """
    if verbose:
        print(f"\n🚀 Training ensemble of {n_models} models with Deep Attention...")

    # Prepare data (same for all models)
    train_df_grouped = create_room_groups(train_df)
    train_vector_df = create_beacon_count_vectors(train_df_grouped)
    X_train_seq, y_train_labels = create_sequences_from_groups(train_vector_df, max_length=50)

    # Encode labels
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train_labels)

    if verbose:
        print(f"  Number of classes: {len(label_encoder.classes_)}")
        print(f"  Classes: {list(label_encoder.classes_)}")
        print(f"  Number of training sequences: {len(X_train_seq)}")

    # Pad sequences
    X_train_padded = pad_sequences(X_train_seq, maxlen=50, padding='post', dtype='float32', value=0.0)

    # Compute class weights
    class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

    # Train multiple models
    models = []
    for i in range(n_models):
        model_seed = base_seed + i * 1000  # 42, 1042, 2042, 3042, 4042
        set_seeds(model_seed)

        if verbose:
            print(f"\n  Training Model {i+1}/{n_models} (seed {model_seed})...")

        # BUILD DEEP ATTENTION MODEL
        model = build_bidirectional_gru_model_with_deep_attention(
            input_shape=(50, 23),
            num_classes=len(label_encoder.classes_)
        )

        # Callbacks
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, verbose=0, min_lr=1e-6)

        # Train
        history = model.fit(
            X_train_padded, y_train,
            epochs=30,
            batch_size=32,
            class_weight=class_weights,
            callbacks=[early_stop, reduce_lr],
            verbose=1 if verbose else 0
        )

        models.append(model)

        if verbose:
            print(f"  ✓ Model {i+1} training completed")

    return models, label_encoder

print("✅ Ensemble training function defined")

✅ Ensemble training function defined


In [9]:
def predict_single_direction(models, sequences, max_seq_length=50):
    """
    Get ensemble predictions for a single direction

    Returns:
        ensemble_proba: (n_samples, n_classes) averaged probability matrix
    """
    # Pad sequences
    X_padded = pad_sequences(sequences, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

    # Get predictions from all models
    all_predictions = []
    for model in models:
        proba = model.predict(X_padded, verbose=0)
        all_predictions.append(proba)

    # Average probabilities across ensemble
    ensemble_proba = np.mean(all_predictions, axis=0)

    return ensemble_proba

def combine_directional_predictions(direction_results, method='confidence_weighted'):
    """
    Combine predictions from multiple directions using confidence weighting
    Now handles 7 directions

    Args:
        direction_results: Dict with keys for all 7 directions
                          Each value is a dict with 'proba' and 'indices'
        method: 'confidence_weighted', 'equal', or 'softmax'

    Returns:
        combined_proba: (n_positions, n_classes) final probability matrix
        position_map: mapping from (date, position) to array index
    """
    # Build a mapping of all unique positions
    all_positions = set()
    direction_names = ['backward_10', 'centered_10', 'forward_10',
                      'backward_15', 'forward_15',
                      'asymm_past', 'asymm_future']

    for direction in direction_names:
        all_positions.update(direction_results[direction]['indices'])

    # Sort positions for consistent ordering
    all_positions = sorted(all_positions)
    position_map = {pos: idx for idx, pos in enumerate(all_positions)}

    # Get number of classes from first available direction
    n_classes = direction_results['backward_10']['proba'].shape[1]
    n_positions = len(all_positions)

    # Initialize combined predictions
    combined_proba = np.zeros((n_positions, n_classes))
    position_counts = np.zeros(n_positions)  # Track how many directions contributed

    # For each direction, add its weighted contribution
    for direction_name in direction_names:
        direction_data = direction_results[direction_name]
        proba = direction_data['proba']
        indices = direction_data['indices']

        # Get confidence (max probability) for each prediction
        confidences = np.max(proba, axis=1)

        # Add weighted contribution to combined predictions
        for i, pos in enumerate(indices):
            pos_idx = position_map[pos]

            if method == 'confidence_weighted':
                # Weight by confidence
                weight = confidences[i]
                combined_proba[pos_idx] += proba[i] * weight
            elif method == 'equal':
                # Equal weight
                combined_proba[pos_idx] += proba[i]
            elif method == 'softmax':
                # Will apply softmax later
                combined_proba[pos_idx] += proba[i] * confidences[i]

            position_counts[pos_idx] += 1 if method == 'equal' else confidences[i]

    # Normalize by total weight
    for i in range(n_positions):
        if position_counts[i] > 0:
            combined_proba[i] /= position_counts[i]

    return combined_proba, position_map

def apply_confidence_weighted_voting(predictions_proba, vote_window=5):
    """
    Confidence-weighted temporal voting

    Instead of simple majority voting, weight each prediction by its confidence (max probability).

    Args:
        predictions_proba: (n_samples, n_classes) probability matrix from ensemble
        vote_window: window size for voting

    Returns:
        voted_predictions: (n_samples,) final class predictions
    """
    n_samples, n_classes = predictions_proba.shape
    voted_predictions = np.zeros(n_samples, dtype=int)

    for i in range(n_samples):
        # Get window boundaries
        half_window = vote_window // 2
        start = max(0, i - half_window)
        end = min(n_samples, i + half_window + 1)

        # Get probabilities within window
        window_proba = predictions_proba[start:end]  # (window_size, n_classes)

        # Get confidence (max probability) for each prediction in window
        window_confidences = np.max(window_proba, axis=1)  # (window_size,)

        # Weight each prediction by its confidence
        weighted_votes = np.zeros(n_classes)
        for j in range(len(window_proba)):
            # Each timestep contributes its probability * its confidence
            weighted_votes += window_proba[j] * window_confidences[j]

        # Final prediction: class with highest weighted vote
        voted_predictions[i] = np.argmax(weighted_votes)

    return voted_predictions

print("✅ Prediction and combination functions defined")

✅ Prediction and combination functions defined


In [10]:
print("="*80)
print("LOADING FULL TRAINING DATA")
print("="*80)

# Load training data
train_df = pd.read_csv(train_data_path)

# Handle timezone in timestamp
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], utc=True)
# Convert to timezone-naive for consistency
train_df['timestamp'] = train_df['timestamp'].dt.tz_localize(None)

print(f"\n✓ Training data loaded")
print(f"  Shape: {train_df.shape}")
print(f"  Columns: {list(train_df.columns)}")
print(f"  Number of unique rooms: {train_df['room'].nunique()}")
print(f"  Unique rooms: {sorted(train_df['room'].unique())}")
print(f"  MAC addresses range: {train_df['mac address'].min()} to {train_df['mac address'].max()}")
print(f"\nFirst few rows:")
print(train_df.head())

LOADING FULL TRAINING DATA

✓ Training data loaded
  Shape: (1099957, 4)
  Columns: ['timestamp', 'mac address', 'RSSI', 'room']
  Number of unique rooms: 22
  Unique rooms: ['501', '502', '503', '505', '506', '508', '510', '511', '512', '513', '515', '516', '517', '518', '520', '522', '523', 'cafeteria', 'cleaning', 'hallway', 'kitchen', 'nurse station']
  MAC addresses range: 1 to 23

First few rows:
            timestamp  mac address  RSSI     room
0 2023-04-10 05:21:46            6   -93  kitchen
1 2023-04-10 05:21:46            6   -93  kitchen
2 2023-04-10 05:21:46            6   -93  kitchen
3 2023-04-10 05:21:46            6   -93  kitchen
4 2023-04-10 05:21:46            6   -93  kitchen


In [11]:
print("\n" + "="*80)
print("TRAINING ENSEMBLE MODELS ON FULL DATASET")
print("="*80)

# Clear any previous sessions
tf.keras.backend.clear_session()

# Train ensemble with seeds: 42, 1042, 2042, 3042, 4042
models, label_encoder = train_ensemble_models(
    train_df,
    n_models=5,
    base_seed=42,
    verbose=True
)

print("\n" + "="*80)
print("✅ ALL MODELS TRAINED SUCCESSFULLY!")
print("="*80)


TRAINING ENSEMBLE MODELS ON FULL DATASET

🚀 Training ensemble of 5 models with Deep Attention...
  Number of classes: 22
  Classes: [np.str_('501'), np.str_('502'), np.str_('503'), np.str_('505'), np.str_('506'), np.str_('508'), np.str_('510'), np.str_('511'), np.str_('512'), np.str_('513'), np.str_('515'), np.str_('516'), np.str_('517'), np.str_('518'), np.str_('520'), np.str_('522'), np.str_('523'), np.str_('cafeteria'), np.str_('cleaning'), np.str_('hallway'), np.str_('kitchen'), np.str_('nurse station')]
  Number of training sequences: 309

  Training Model 1/5 (seed 42)...
Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.0385 - loss: 3.2153 - learning_rate: 0.0010
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1283 - loss: 3.1567 - learning_rate: 0.0010
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1757 - loss: 3.0604 - learning_rate: 0.0010
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1728 - loss: 2.9269 - 

In [13]:
print("\n" + "="*80)
print("SAVING MODELS AND ENCODER")
print("="*80)

# Create directory for models
model_dir = '/content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models'
os.makedirs(model_dir, exist_ok=True)

# Save each model
for i, model in enumerate(models):
    seed = 42 + i * 1000
    model_path = os.path.join(model_dir, f'model_seed_{seed}.keras')
    model.save(model_path)
    print(f"  ✓ Model {i+1} (seed {seed}) saved to: {model_path}")

# Save label encoder
encoder_path = os.path.join(model_dir, 'label_encoder.pkl')
with open(encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)
print(f"\n  ✓ Label encoder saved to: {encoder_path}")

# Save class names for reference
classes_path = os.path.join(model_dir, 'class_names.txt')
with open(classes_path, 'w') as f:
    f.write("Room Classes:\n")
    f.write("=============\n")
    for i, class_name in enumerate(label_encoder.classes_):
        f.write(f"{i}: {class_name}\n")
print(f"  ✓ Class names saved to: {classes_path}")

print("\n" + "="*80)
print("✅ ALL MODELS AND ENCODER SAVED!")
print("="*80)


SAVING MODELS AND ENCODER
  ✓ Model 1 (seed 42) saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/model_seed_42.keras
  ✓ Model 2 (seed 1042) saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/model_seed_1042.keras
  ✓ Model 3 (seed 2042) saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/model_seed_2042.keras
  ✓ Model 4 (seed 3042) saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/model_seed_3042.keras
  ✓ Model 5 (seed 4042) saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/model_seed_4042.keras

  ✓ Label encoder saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/label_encoder.pkl
  ✓ Class names saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/trained_models/class_names.txt

✅ ALL MODELS AND ENCODER SAVED!


In [14]:
print("\n" + "="*80)
print("LOADING REAL TEST DATA")
print("="*80)

# Load test data
test_df_original = pd.read_csv(test_data_path)

# Handle timestamp format (no timezone in test data)
test_df_original['timestamp'] = pd.to_datetime(test_df_original['timestamp'])

print(f"\n✓ Test data loaded")
print(f"  Shape: {test_df_original.shape}")
print(f"  Columns: {list(test_df_original.columns)}")
print(f"  Unique timestamps: {test_df_original['timestamp'].nunique()}")
print(f"  MAC addresses range: {test_df_original['mac address'].min()} to {test_df_original['mac address'].max()}")
print(f"\nFirst few rows:")
print(test_df_original.head())

# Keep a copy with original index for later
test_df_original['original_index'] = test_df_original.index


LOADING REAL TEST DATA

✓ Test data loaded
  Shape: (62222, 6)
  Columns: ['Unnamed: 0', 'user_id', 'timestamp', 'mac address', 'RSSI', 'power']
  Unique timestamps: 5721
  MAC addresses range: 1 to 23

First few rows:
   Unnamed: 0  user_id           timestamp  mac address  RSSI       power
0         766       90 2023-04-14 10:01:40            7   -97 -2147483648
1         764       90 2023-04-14 10:01:40            7   -97 -2147483648
2         765       90 2023-04-14 10:01:40            7   -97 -2147483648
3         762       90 2023-04-14 10:01:40            7   -97 -2147483648
4         761       90 2023-04-14 10:01:40            7   -97 -2147483648


In [15]:
print("\n" + "="*80)
print("MAKING PREDICTIONS ON TEST DATA")
print("="*80)

# For prediction, we need a 'room' column (can be dummy since we don't have labels)
# We'll use a placeholder value
test_df = test_df_original.copy()
test_df['room'] = 'unknown'  # Placeholder

# Create beacon vectors from test data
print("\n  Creating beacon count vectors...")
test_vectors = create_beacon_count_vectors(test_df)
print(f"  ✓ Created {len(test_vectors)} 1-second windows")

# Create multi-directional windows
print("\n  Creating 7-directional windows...")
direction_windows = create_extended_multidirectional_windows(test_vectors)

for direction_name in ['backward_10', 'centered_10', 'forward_10', 'backward_15', 'forward_15', 'asymm_past', 'asymm_future']:
    n_windows = len(direction_windows[direction_name]['sequences'])
    print(f"    {direction_name:15s}: {n_windows:6d} windows")

# Get predictions for each direction
print("\n  Getting directional predictions...")
direction_results = {}
direction_names = ['backward_10', 'centered_10', 'forward_10',
                  'backward_15', 'forward_15',
                  'asymm_past', 'asymm_future']

for direction_name in direction_names:
    print(f"    Predicting {direction_name}...", end=" ")

    sequences = direction_windows[direction_name]['sequences']
    proba = predict_single_direction(models, sequences, max_seq_length=50)

    direction_results[direction_name] = {
        'proba': proba,
        'indices': direction_windows[direction_name]['indices'],
        'labels': direction_windows[direction_name]['labels']
    }

    avg_conf = np.mean(np.max(proba, axis=1))
    print(f"avg confidence: {avg_conf:.3f}")

# Combine directional predictions
print("\n  Combining 7 directions using confidence weighting...")
combined_proba, position_map = combine_directional_predictions(
    direction_results,
    method='confidence_weighted'
)
print(f"  ✓ Combined predictions for {len(position_map)} unique positions")

# Apply temporal voting
print("\n  Applying temporal voting (window=5)...")
y_pred_voted_encoded = apply_confidence_weighted_voting(combined_proba, vote_window=5)
print(f"  ✓ Voting completed")

# Decode predictions
print("\n  Decoding predictions to room names...")
y_pred = label_encoder.inverse_transform(y_pred_voted_encoded)
print(f"  ✓ Predictions decoded")

print("\n" + "="*80)
print("✅ PREDICTIONS COMPLETED!")
print("="*80)


MAKING PREDICTIONS ON TEST DATA

  Creating beacon count vectors...
  ✓ Created 5721 1-second windows

  Creating 7-directional windows...
    backward_10    :   5712 windows
    centered_10    :   5712 windows
    forward_10     :   5712 windows
    backward_15    :   5707 windows
    forward_15     :   5707 windows
    asymm_past     :   5707 windows
    asymm_future   :   5707 windows

  Getting directional predictions...
    Predicting backward_10... avg confidence: 0.531
    Predicting centered_10... avg confidence: 0.531
    Predicting forward_10... avg confidence: 0.531
    Predicting backward_15... avg confidence: 0.572
    Predicting forward_15... avg confidence: 0.572
    Predicting asymm_past... avg confidence: 0.572
    Predicting asymm_future... avg confidence: 0.572

  Combining 7 directions using confidence weighting...
  ✓ Combined predictions for 5721 unique positions

  Applying temporal voting (window=5)...
  ✓ Voting completed

  Decoding predictions to room names.

In [16]:
print("\n" + "="*80)
print("PROPAGATING PREDICTIONS TO ORIGINAL DATA")
print("="*80)

# Create a mapping from (date, position) to predicted room
prediction_map = {}
for pos, idx in position_map.items():
    prediction_map[pos] = y_pred[idx]

print(f"\n  Created prediction map for {len(prediction_map)} positions")

# Now we need to map back to original timestamps
# First, recreate the position mapping for test_vectors
test_vectors_with_date = test_vectors.copy()
test_vectors_with_date['dt'] = pd.to_datetime(test_vectors_with_date['timestamp'])
test_vectors_with_date['date'] = test_vectors_with_date['dt'].dt.date

# Create position index for each timestamp within each day
predictions_by_timestamp = {}

for date, day_group in test_vectors_with_date.groupby('date'):
    day_group = day_group.sort_values('timestamp').reset_index(drop=True)
    for i, row in day_group.iterrows():
        pos_key = (date, i)
        if pos_key in prediction_map:
            timestamp = row['timestamp']
            predicted_room = prediction_map[pos_key]
            predictions_by_timestamp[timestamp] = predicted_room

print(f"  Mapped predictions to {len(predictions_by_timestamp)} unique timestamps")

# Assign predictions to original test data
print("\n  Assigning predictions to original rows...")
test_df_final = test_df_original.copy()
test_df_final['Location'] = test_df_final['timestamp'].map(predictions_by_timestamp)

# Check for any missing predictions
missing_predictions = test_df_final['Location'].isna().sum()
if missing_predictions > 0:
    print(f"  ⚠️  Warning: {missing_predictions} rows have no predictions (edge cases)")
    print(f"     These will be forward-filled with nearest prediction")
    # Forward fill missing values
    test_df_final['Location'] = test_df_final['Location'].fillna(method='ffill')
    # If still any NaN (at the beginning), backward fill
    test_df_final['Location'] = test_df_final['Location'].fillna(method='bfill')

print(f"\n  ✓ All {len(test_df_final)} rows have predictions")

# Show prediction distribution
print("\n  Prediction distribution:")
print(test_df_final['Location'].value_counts().sort_index())

print("\n" + "="*80)
print("✅ PROPAGATION COMPLETED!")
print("="*80)


PROPAGATING PREDICTIONS TO ORIGINAL DATA

  Created prediction map for 5721 positions
  Mapped predictions to 5721 unique timestamps

  Assigning predictions to original rows...

  ✓ All 62222 rows have predictions

  Prediction distribution:
Location
501                585
502                276
503                990
506               1828
508                546
510                 43
511                143
513                340
516                264
517                144
518                798
520                746
522                687
523                935
cafeteria        10330
cleaning          5505
hallway            174
kitchen          18687
nurse station    19201
Name: count, dtype: int64

✅ PROPAGATION COMPLETED!


In [17]:
print("\n" + "="*80)
print("SAVING FINAL PREDICTIONS")
print("="*80)

# Drop the helper columns and keep original columns + Location
output_df = test_df_final.drop(columns=['original_index'], errors='ignore')

# Save to the specified filename
output_path = '/content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/predicted_BLE_Test_Data.csv'
output_df.to_csv(output_path, index=False)

print(f"\n  ✓ Predictions saved to: {output_path}")
print(f"  Shape: {output_df.shape}")
print(f"  Columns: {list(output_df.columns)}")

print("\n  Preview of saved data:")
print(output_df.head(10))

print("\n  Final statistics:")
print(f"    Total rows: {len(output_df)}")
print(f"    Unique timestamps: {output_df['timestamp'].nunique()}")
print(f"    Unique locations predicted: {output_df['Location'].nunique()}")
print(f"    Locations: {sorted(output_df['Location'].unique())}")

print("\n" + "="*80)
print("🎉 DEPLOYMENT PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)
print("\nSummary:")
print("  ✅ Trained 5 ensemble models on full training data")
print("  ✅ Saved all models and label encoder")
print("  ✅ Made predictions on real test data")
print("  ✅ Propagated window-level predictions to original rows")
print("  ✅ Saved final predictions to predicted_BLE_Test_Data.csv")
print("\n" + "="*80)


SAVING FINAL PREDICTIONS

  ✓ Predictions saved to: /content/drive/MyDrive/ABC_Full_Train_Data_And_Real_Test_Data/predicted_BLE_Test_Data.csv
  Shape: (62222, 7)
  Columns: ['Unnamed: 0', 'user_id', 'timestamp', 'mac address', 'RSSI', 'power', 'Location']

  Preview of saved data:
   Unnamed: 0  user_id           timestamp  mac address  RSSI       power  \
0         766       90 2023-04-14 10:01:40            7   -97 -2147483648   
1         764       90 2023-04-14 10:01:40            7   -97 -2147483648   
2         765       90 2023-04-14 10:01:40            7   -97 -2147483648   
3         762       90 2023-04-14 10:01:40            7   -97 -2147483648   
4         761       90 2023-04-14 10:01:40            7   -97 -2147483648   
5         763       90 2023-04-14 10:01:40            7   -97 -2147483648   
6         777       90 2023-04-14 10:01:42            7   -91 -2147483648   
7         783       90 2023-04-14 10:01:42            7   -91 -2147483648   
8         782       90 2